## 3.3 通过自注意力机制关注输入的不同部分

In [19]:
import torch

In [20]:
inputs = torch.tensor(
    [
        [0.43, 0.15, 0.89],   # Your
        [0.55, 0.87, 0.66],   # journey
        [0.57, 0.85, 0.64],   # starts
        [0.22, 0.58, 0.33],   # with
        [0.77, 0.25, 0.10],   # one
        [0.05, 0.80, 0.55],   # step
    ]
)

In [21]:
query = inputs[1]

# 使用点积计算所有输入与 inputs[1] 的注意力分数
attn_score_2 = torch.empty(inputs.shape[0])
for i, x_i in enumerate(inputs):
    attn_score_2[i] = torch.dot(x_i, query)
print(attn_score_2)

tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])


In [22]:
def softmax_naive(x):
    return torch.exp(x) / torch.exp(x).sum(dim=0)

# 注意力权重归一化
attn_weights_2 = softmax_naive(attn_score_2)
print("Attention weights:", attn_weights_2)
print("Sum:", attn_weights_2.sum())

Attention weights: tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
Sum: tensor(1.)


In [23]:
query = inputs[1]
print("query.shape:", query.shape)
context_vec_2 = torch.zeros(query.shape)
print("context_vec_2:", context_vec_2)

# 计算上下文向量：将所有输入的向量与其对应的注意力权重相乘而获得
for i, x_i in enumerate(inputs):
    context_vec_2 += attn_weights_2[i] * x_i
print("context_vec_2:", context_vec_2)

query.shape: torch.Size([3])
context_vec_2: tensor([0., 0., 0.])
context_vec_2: tensor([0.4419, 0.6515, 0.5683])


### 推广一下，计算所有输入的上下文向量

1. 计算注意力分数：衡量每个词对其他词的"相关性"或"相似度"
    - 计算每对词之间的点积，得到相似度分数
    - 分数越高，表示两个词越相关
    - 为后续的加权平均做准备
2. 归一化获取注意力权重：将原始分数转换为概率分布，确保权重和为1
    - 使用 softmax 将任意实数转换为 0-1 之间的概率
    - 每行（每个词）对所有词的注意力权重总和为1
    - 这样每个词对其他词的"关注度"就变成了百分比
3. 计算上下文向量：根据注意力权重，为每个词生成包含上下文信息的向量表示
    - 将每个词对所有词的注意力权重作为系数
    - 对所有词的向量进行加权平均
    - 得到每个词的"上下文向量"，包含了整个序列的信息

这3个步骤实现了自注意力机制的核心思想：

- "看" - 计算每个词对其他词的关注度
- "权衡" - 将关注度转换为权重
- "融合" - 根据权重融合所有词的信息

最终效果： 每个词的向量表示都包含了整个序列的上下文信息，而不仅仅是自己的信息。这样模型就能理解词与词之间的关系，比如"journey starts"中的"starts"会更多地关注"journey"的信息。

In [25]:
# 1. 计算注意力分数
attn_scores = torch.empty(6, 6)
for i, x_i in enumerate(inputs):
    for j, x_j in enumerate(inputs):
        attn_scores[i, j] = torch.dot(x_i, x_j)
print(attn_scores)

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


In [30]:
# 2. 归一化获取注意力权重
attn_weights = torch.softmax(attn_scores, dim=-1)
print(attn_weights)

# -1 表示在最后一个维度进行归一化，因为 attn_scores 是一个 [行, 列]，所以这里是在列上进行归一化，使得每行的值（在列维度的总和）为 1
# "沿着列的方向" = 在每一行内部，从左到右（列 0 到列 5）进行归一化
# 每一行都独立进行这个过程，结果每一行的 6 个数字加起来都等于 1
row_2_sum = sum([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
print("Row 2 sum:", row_2_sum)
print("All row sums:", attn_weights.sum(dim=-1))

tensor([[0.2098, 0.2006, 0.1981, 0.1242, 0.1220, 0.1452],
        [0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581],
        [0.1390, 0.2369, 0.2326, 0.1242, 0.1108, 0.1565],
        [0.1435, 0.2074, 0.2046, 0.1462, 0.1263, 0.1720],
        [0.1526, 0.1958, 0.1975, 0.1367, 0.1879, 0.1295],
        [0.1385, 0.2184, 0.2128, 0.1420, 0.0988, 0.1896]])
Row 2 sum: 1.0
All row sums: tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000])


In [31]:
# 3. 计算上下文向量
all_contexts_vec = attn_weights @ inputs
print(all_contexts_vec)

tensor([[0.4421, 0.5931, 0.5790],
        [0.4419, 0.6515, 0.5683],
        [0.4431, 0.6496, 0.5671],
        [0.4304, 0.6298, 0.5510],
        [0.4671, 0.5910, 0.5266],
        [0.4177, 0.6503, 0.5645]])
